In [1]:
from bs4 import BeautifulSoup
import re
import urllib2

In [2]:
nPerPage = 72
nPages = 83

##### Getting the product urls

In [92]:
def getsoup(url):
    for attempt in range(5):
        try:
            raw_page = urllib2.urlopen(url).read()
        except:
            continue
        break
    else: #executed if for loop never breaks, i.e. never connects to url
        print "Didn't get {}".format(url)
        return None
    return BeautifulSoup(raw_page)

In [7]:
mainurl = "http://www.drugstore.com/templates/gn/default.asp?catid=180646&it=5951&"
producturls = []
f = open("product_urls.txt","wb")
for i in xrange(nPages):
    offset = (i+1)*nPerPage
    listurl = mainurl+"&Nao="+str(offset)+"&ipp=72&N=0"
    soup = getsoup(listurl)
    links = soup.select("a.oesLink")
    for link in links:
        if link.find('span')['class'][0] != "verticalFeaturedProductText":
            producturls.append("www.drugstore.com"+link["href"])
            f.write("www.drugstore.com"+link["href"]+"\n")
f.close()

##### Scraping product pages

In [129]:
def getName(sp):
    return sp.select('div#divPGContainer h1.captionText')[0].text

def getPrice(sp):
    tag = sp.select('div#productprice span.price')
    tagOld = sp.select('div#OldPriceForUnavailableProductDiv span')
    if tag:
        price = re.search(PRICE_RE,tag[0].text).group(1)
    elif tagOld:
        price = re.search(PRICEOLD_RE,tagOld[0].text).group(1)
    else:
        price = -999
    return float(price)

def getCategories(sp):
    trivial = ['home','beauty','skin care']
    cats = [cat.text for cat in sp.select('a.breadcrumb') if cat.text not in trivial]
    return cats

def getIngredients(sp):
    tag = sp.select("div#divingredientsPDetail td.contenttd")
    if tag:
        return tag[0].text.split(', ')
    else:
        return []

def getRating(sp):
    tag = sp.select("div.pr-snapshot-rating span.average")
    if tag:
        return float(tag[0].text)
    else:
        return -999.0

def getNReviews(sp):
    tag = sp.select("span.count")
    if tag:
        return int(tag[0].text)
    else:
        return -999
    
def getPros(sp):
    pros = sp.select('div.pr-attribute-pros ul.pr-snapshot-attribute-value-list li')
    return [pro.text for pro in pros]

def getCons(sp):
    cons = sp.select('div.pr-attribute-cons ul.pr-snapshot-attribute-value-list li')
    return [con.text for con in cons]

In [121]:
f = open("missedproducts_3.txt","rb")
producturls = f.read().splitlines()
f.close()

In [131]:
PRICE_RE = re.compile('^\$([0-9]+\.[0-9]{2})')
PRICEOLD_RE = re.compile('\$([0-9]+\.[0-9]{2})')
import json
outfile = open('productJSON_4.txt', 'wb')
logfile = open('missedproducts_4.txt','wb')
for prod in producturls[:1]:
    #print prod
    prodinfo = {}
    soup = getsoup("http://"+prod)
    #print soup
    if soup:
        try:
            prodinfo['name'] = getName(soup)
            prodinfo['price'] = getPrice(soup)
            prodinfo['categories'] = getCategories(soup)
            prodinfo['ingredients'] = getIngredients(soup)
            prodinfo['rating'] = getRating(soup)
            prodinfo['nreviews'] = getNReviews(soup)
            prodinfo['pros'] = getPros(soup)
            prodinfo['cons'] = getCons(soup)
            json_str = json.dumps(prodinfo)
            outfile.write(json_str+'\n')
            #print json_str
            #print "Success."
        except:
            logfile.write(prod+'\n')
            #print "Failed"
    else:
        logfile.write(prod+'\n')
        #pass
logfile.close()
outfile.close()

In [81]:
#f = open("productJSON.txt",'rb')
#jsons = f.read().splitlines()

##### pandas


In [132]:
import pandas as pd

In [134]:
with open('AllProducts071715.txt', 'rb') as f:
    data = f.read().splitlines()
data_json_str = "[" + ','.join(data) + "]"
df = pd.read_json(data_json_str)

In [198]:
df.head().to_html()

u'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>categories</th>\n      <th>cons</th>\n      <th>ingredients</th>\n      <th>name</th>\n      <th>nreviews</th>\n      <th>price</th>\n      <th>pros</th>\n      <th>rating</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td> [baby &amp; mom, bath time &amp; skin care, toys &amp; acc...</td>\n      <td>                                                []</td>\n      <td> []</td>\n      <td>       Baby Buddy Natural Bath Sponge, Yellow\xa01 ea</td>\n      <td>   2</td>\n      <td>  6.99</td>\n      <td>                                                []</td>\n      <td> 4.5</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>                                 [skin care tools]</td>\n      <td>                                                []</td>\n      <td> []</td>\n      <td> Amope Pedi Perfect Electronic Pedicure Foot Fi...</td>\n      <td>  27</t

In [201]:
set([item for sublist in df.categories[:10] for item in sublist])

{u'acne',
 u'all moisturizers',
 u'all sunscreen',
 u'baby & mom',
 u'bar soap',
 u'bath time & skin care',
 u'cleansers',
 u'eye treatments',
 u'first aid',
 u'medicine & health',
 u'moisturizers',
 u'scar & wound',
 u'skin care sets',
 u'skin care tools',
 u'skin treatments & peels',
 u'solution kits',
 u'sun care',
 u'toys & accessories'}

In [189]:
facestuff = PickCategories(df,['face scrubs','face wash','acne'])
noalc = facestuff[['Stearyl Alcohol' not in li for li in facestuff.ingredients]][['name','price','rating','nreviews','ingredients']]


In [188]:
df[map(df.price,lambda x: int(x)) > 5]


TypeError: argument 2 to map() must support iteration

In [190]:
noalc.sort(columns=['rating','nreviews','price'], ascending=[0,0,1])

,name,price,rating,nreviews,ingredients
3861,Shea Terra Organics Rose Hips Black Soap Deep ...,14.40,5.0,7,"[Deionized Water, Yoruba Black Soap, Cocoa Pod..."
4044,"Raw Materials Skin Grit Lathering Face Scrub, ...",15.99,5.0,7,"[Water, Aloe Vera (Aloe Barbadensis) Leaf Juic..."
4164,OXY Daily Defense Ultra Facial Cleanser 5 oz (...,5.99,5.0,4,[Active Ingredients: Salicylic Acid 2% (Acne T...
3809,University Medical AcneFree Energizing 3-in-1 ...,5.59,5.0,1,[Active Ingredients: Salicylic Acid - 0.5 % (A...
3364,Clearasil Daily Clear Hydra-Blast Oil-Free Pad...,6.49,5.0,1,[Active Ingredients: Salicylic Acid (2%) (Acne...
5689,MyChelle Clear Skin Cleanser 2.1 Ounces,11.25,5.0,1,"[Aqua (Water), Sodium Laurylglucosides Hydroxy..."
4,Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),5.79,4.8,12,[]
2198,Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),5.79,4.8,12,[]
2972,Neutrogena Oil-Free Acne Stress Control Night ...,6.79,4.8,5,"[Active Ingredients: Salicylic Acid (2%, Acne ..."
2928,"Every Man Jack Face Scrub and Pre-Shave, Signa...",4.79,4.7,15,"[Water, Cocamidopropyl Betaine, Polyethylene, ..."


In [208]:
def PickCategories(d, mycats):
    if not isinstance(mycats, list):
        mycats = [mycats]
    bools = []
    for catlist in d.categories:
        if len(list(set(catlist)&set(mycats))) == 0:
            bools.append(False)
        else:
            bools.append(True)
    return d[bools]

In [202]:
facestuff

,categories,cons,ingredients,name,nreviews,price,pros,rating
4,"[cleansers, acne]",[],[],Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),12,5.79,"[Cleans effectively (12), Easy to use (11), Pl...",4.8
147,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,Clearasil Daily Clear Hydra-Blast Oil-Free Fac...,-999,6.49,[],-999.0
195,"[men's skin care, face wash]",[],"[Sodium Lauroyl Isethionate, Stearic Acid, Sod...","Dove Men+Care Extra Fresh Body + Face Bar, 4 o...",-999,15.19,[],-999.0
382,"[men's skin care, face wash]",[],"[Water (Aqua), Glycerin, Sodium Lauroyl Isethi...","Dove Men+Care Face Wash, Hydrate+ 5 fl oz (148...",-999,6.09,[],-999.0
406,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide - 2.5 % ...,University Medical AcneFree Energizing Acne Wa...,-999,6.39,[],-999.0
436,"[men's skin care, face wash]",[],"[Water, Sodium Laureth Sulfate, Petrolatum, Ac...","Dove Men+Care Body & Face Wash, Hydration Bala...",-999,5.79,[],-999.0
467,"[men's skin care, face wash]",[],"[Water, Sodium Laureth 3 Sulfate, Cocamide DEA...",Bald Guyz Daily Wash 4 fl oz (118 ml),-999,5.99,[],-999.0
514,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%) (Acne...,Clearasil Ultra Rapid Action Daily Gel Wash 6....,-999,8.99,[],-999.0
559,"[cleansers, acne]",[],"[Water, Alcohol, Sorbitol, Propanediol, Decyl ...",Olay Fresh Effects Clear Skin See Spot Swipe O...,-999,4.50,[],-999.0
586,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid 0.5% (Acne...,Clear Start Breakout Clearing Foaming Wash 6 f...,-999,18.99,[],-999.0


In [209]:
PickCategories(df,str(u'acne'))

,categories,cons,ingredients,name,nreviews,price,pros,rating
4,"[cleansers, acne]",[],[],Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),12,5.79,"[Cleans effectively (12), Easy to use (11), Pl...",4.8
147,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,Clearasil Daily Clear Hydra-Blast Oil-Free Fac...,-999,6.49,[],-999.0
406,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide - 2.5 % ...,University Medical AcneFree Energizing Acne Wa...,-999,6.39,[],-999.0
514,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%) (Acne...,Clearasil Ultra Rapid Action Daily Gel Wash 6....,-999,8.99,[],-999.0
559,"[cleansers, acne]",[],"[Water, Alcohol, Sorbitol, Propanediol, Decyl ...",Olay Fresh Effects Clear Skin See Spot Swipe O...,-999,4.50,[],-999.0
586,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid 0.5% (Acne...,Clear Start Breakout Clearing Foaming Wash 6 f...,-999,18.99,[],-999.0
600,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide (10%) (A...,Neutrogena Rapid Clear Neutrogena Rapid Clear ...,-999,8.99,[],-999.0
637,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"Dial Deep Cleansing Face Wash, Oil Free Acne C...",-999,5.49,[],-999.0
652,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,"Walgreens Acne Wash, Oil Free 6 fl oz",-999,5.49,[],-999.0
658,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"OXY Daily Defense Advanced Facial Cleanser, Al...",-999,5.79,[],-999.0


In [210]:
df_show = df.copy()

In [212]:
PickCategories(df_show,str(u'acne'))

,categories,cons,ingredients,name,nreviews,price,pros,rating
4,"[cleansers, acne]",[],[],Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),12,5.79,"[Cleans effectively (12), Easy to use (11), Pl...",4.8
147,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,Clearasil Daily Clear Hydra-Blast Oil-Free Fac...,-999,6.49,[],-999.0
406,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide - 2.5 % ...,University Medical AcneFree Energizing Acne Wa...,-999,6.39,[],-999.0
514,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%) (Acne...,Clearasil Ultra Rapid Action Daily Gel Wash 6....,-999,8.99,[],-999.0
559,"[cleansers, acne]",[],"[Water, Alcohol, Sorbitol, Propanediol, Decyl ...",Olay Fresh Effects Clear Skin See Spot Swipe O...,-999,4.50,[],-999.0
586,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid 0.5% (Acne...,Clear Start Breakout Clearing Foaming Wash 6 f...,-999,18.99,[],-999.0
600,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide (10%) (A...,Neutrogena Rapid Clear Neutrogena Rapid Clear ...,-999,8.99,[],-999.0
637,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"Dial Deep Cleansing Face Wash, Oil Free Acne C...",-999,5.49,[],-999.0
652,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,"Walgreens Acne Wash, Oil Free 6 fl oz",-999,5.49,[],-999.0
658,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"OXY Daily Defense Advanced Facial Cleanser, Al...",-999,5.79,[],-999.0


In [218]:
df_show = PickCategories(df_show,'acne')

In [219]:
df_show

,categories,cons,ingredients,name,nreviews,price,pros,rating,test
4,"[cleansers, acne]",[],[],Phisoderm Anti-Blemish Body Wash 10 oz (295 ml),12,5.79,"[Cleans effectively (12), Easy to use (11), Pl...",4.8,5.79
147,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,Clearasil Daily Clear Hydra-Blast Oil-Free Fac...,-999,6.49,[],-999.0,6.49
406,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide - 2.5 % ...,University Medical AcneFree Energizing Acne Wa...,-999,6.39,[],-999.0,6.39
514,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%) (Acne...,Clearasil Ultra Rapid Action Daily Gel Wash 6....,-999,8.99,[],-999.0,8.99
559,"[cleansers, acne]",[],"[Water, Alcohol, Sorbitol, Propanediol, Decyl ...",Olay Fresh Effects Clear Skin See Spot Swipe O...,-999,4.50,[],-999.0,4.50
586,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid 0.5% (Acne...,Clear Start Breakout Clearing Foaming Wash 6 f...,-999,18.99,[],-999.0,18.99
600,"[cleansers, acne]",[],[Active Ingredients: Benzoyl Peroxide (10%) (A...,Neutrogena Rapid Clear Neutrogena Rapid Clear ...,-999,8.99,[],-999.0,8.99
637,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"Dial Deep Cleansing Face Wash, Oil Free Acne C...",-999,5.49,[],-999.0,5.49
652,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid (2%)Inacti...,"Walgreens Acne Wash, Oil Free 6 fl oz",-999,5.49,[],-999.0,5.49
658,"[cleansers, acne]",[],[Active Ingredients: Salicylic Acid - 2 % (Acn...,"OXY Daily Defense Advanced Facial Cleanser, Al...",-999,5.79,[],-999.0,5.79


In [225]:
def ExcludeIngredient(d, bad_ing):
    if not isinstance(bad_ing,list):
        bad_ing = [bad_ing]
    bools = []
    for i, inglist in enumerate(d.ingredients):
        bools.append(True)
        for ing in inglist:
            for bad in bad_ing:
                if bad.lower() in ing.lower():
                    bools[i]=False
                    break
    return d[bools]